In [ ]:
import os
from openai import OpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

client = OpenAI(api_key="OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

In [2]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

In [3]:
def translate_with_gpt(text, source_lang="ko", target_lang="en"):
    prompt = f"Translate this from {source_lang} to {target_lang}:\n\n{text}"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return response.choices[0].message.content.strip()


In [12]:
translate_with_gpt("문형빈 교수님의 논문 목록은")

"The list of Professor Moon Hyung-bin's papers is"

In [4]:
def classify_question_type(question_ko):
    prompt = f"""
다음 질문의 유형을 아래 중 하나로 분류해 주세요:
- 논문_목록
- 논문_요약
- 연구_흐름

질문: {question_ko}
질문 유형:"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt.strip()}],
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [5]:
# 1. 한글 질문 입력
question_ko = "문형빈 교수님의 논문 목록은"

# 2. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 3. 질문 영어로 번역 (Chroma DB 검색용)
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=30)
context_text = "\n\n".join([doc.page_content for doc in docs])

# 5. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
다음은 교수님의 여러 논문 내용입니다. 아래 질문을 바탕으로 해당 교수의 주요 논문 제목과 발표 연도, 핵심 키워드를 모두 나열해 주세요.

질문: {question}
논문 정보:
{context}

논문 목록 요약:"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
다음은 한 편의 논문 내용입니다. 이 논문의 연구대상, 연구방법, 연구결과, 제언을 간결하고 명확하게 요약해 주세요.

논문 내용:
{context}

요약:"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
다음은 한 교수의 여러 논문 내용입니다. 연구 주제나 관심 분야가 시간에 따라 어떻게 변화했는지 분석하여 시기별 흐름을 요약해 주세요.

질문: {question}
논문 내용:
{context}

연구 흐름 요약:"""
    )
}

# 6. 프롬프트 선택 + LLM 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm  # 최신 LangChain 방식

# 7. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 8. 결과 출력
print(f"[{question_type.upper()}]")
print(result.content)

C:\Users\wonhy\AppData\Local\Temp\ipykernel_21964\3844153672.py:52: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")


[논문_목록]
문형빈 교수님의 주요 논문 목록은 다음과 같습니다:

1. 논문 제목: "Well-to-wheel analysis of greenhouse gas emissions for electric vehicles based on electricity generation mix: a global perspective"
   - 발표 연도: 2017
   - 핵심 키워드: Greenhouse gas emissions, Electric vehicles, Electricity generation mix, Global perspective

2. 논문 제목: "Forecasting the value of battery electric vehicles compared to internal combustion engine vehicles: the influence of driving range and battery technology"
   - 발표 연도: 2020
   - 핵심 키워드: Battery electric vehicles, Driving range, Battery technology, Internal combustion engine vehicles

3. 논문 제목: "Managing spent nuclear fuel in South Korea: Heterogeneous public attitudes toward different management strategies at individual- and segment levels"
   - 발표 연도: 2020
   - 핵심 키워드: Spent nuclear fuel, Public attitudes, Management strategies, South Korea

4. 논문 제목: "Attitudes in Korea toward Introducing Smart Policing Technologies: Differences between the General Public and Police Officers"

In [6]:
# 1. 한글 질문 입력
question_ko = "Well-to-wheel analysis of greenhouse gas emissions for electric vehicles based on electricity generation mix: a global perspective논문을 요약해줘"

# 2. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 3. 질문 영어로 번역 (Chroma DB 검색용)
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=30)
context_text = "\n\n".join([doc.page_content for doc in docs])

# 5. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
다음은 교수님의 여러 논문 내용입니다. 아래 질문을 바탕으로 해당 교수의 주요 논문 제목과 발표 연도, 핵심 키워드를 모두 나열해 주세요.

질문: {question}
논문 정보:
{context}

논문 목록 요약:"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
다음은 한 편의 논문 내용입니다. 이 논문의 연구대상, 연구방법, 연구결과, 제언을 간결하고 명확하게 요약해 주세요.

논문 내용:
{context}

요약:"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
다음은 한 교수의 여러 논문 내용입니다. 연구 주제나 관심 분야가 시간에 따라 어떻게 변화했는지 분석하여 시기별 흐름을 요약해 주세요.

질문: {question}
논문 내용:
{context}

연구 흐름 요약:"""
    )
}

# 6. 프롬프트 선택 + LLM 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm  # 최신 LangChain 방식

# 7. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 8. 결과 출력
print(f"[{question_type.upper()}]")
print(result.content)

[논문_요약]
**연구대상**: 이 논문은 전 세계적으로 증가하는 전기차(EV)의 수요와 그로 인한 전력망에 미치는 영향을 연구 대상으로 삼고 있다. 특히, 전기차의 확산이 전력 수요에 미치는 영향을 분석하고, 이를 통해 전력 공급 인프라의 안정성을 보장하기 위한 방안을 모색한다.

**연구방법**: 연구는 소비자의 전기차 사용 및 충전 패턴을 분석하기 위해 설문조사와 이산 선택 실험을 활용했다. 이를 통해 전기차 시장의 확산 시나리오별 전력 수요를 추정하고, 소비자 선호도를 반영한 전기차 충전 인프라의 최적화를 모색했다.

**연구결과**: 연구 결과, 기술 발전 수준에 따라 전기차의 시장 점유율은 2.31%에서 8.74%까지 증가할 것으로 예상되었다. 또한, 충전 패턴 분석을 통해 소비자들이 주로 낮에 공공 충전소를 이용하며, 피크 시간대에는 빠른 충전을 선호하는 것으로 나타났다. 이러한 결과는 전력망의 안정성을 위해 정부가 충전 인프라를 확장해야 할 필요성을 시사한다.

**제언**: 연구는 전기차 시장 확산에 따른 전력 수요를 보다 정밀하게 추정하기 위해 소비자의 운전 및 충전 패턴을 지속적으로 분석할 것을 제안한다. 또한, EV 충전 인프라의 확장을 위한 정책 마련 시 소비자 선호도를 고려해야 하며, 장기적으로는 전력망의 안정성을 확보하기 위한 정책적 대응이 필요함을 강조한다.


In [7]:
# 1. 한글 질문 입력
question_ko = "문형빈 교수님의 연구 동향 알려줘"

# 2. 질문 유형 분류
question_type = classify_question_type(question_ko).strip()

# 3. 질문 영어로 번역 (Chroma DB 검색용)
question_en = translate_with_gpt(question_ko)

# 4. Chroma DB 검색
docs = db3.similarity_search(question_en, k=30)
context_text = "\n\n".join([doc.page_content for doc in docs])

# 5. 프롬프트 템플릿 정의
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
다음은 교수님의 여러 논문 내용입니다. 아래 질문을 바탕으로 해당 교수의 주요 논문 제목과 발표 연도, 핵심 키워드를 모두 나열해 주세요.

질문: {question}
논문 정보:
{context}

논문 목록 요약:"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
다음은 한 편의 논문 내용입니다. 이 논문의 연구대상, 연구방법, 연구결과, 제언을 간결하고 명확하게 요약해 주세요.

논문 내용:
{context}

요약:"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
다음은 한 교수의 여러 논문 내용입니다. 연구 주제나 관심 분야가 시간에 따라 어떻게 변화했는지 분석하여 시기별 흐름을 요약해 주세요.

질문: {question}
논문 내용:
{context}

연구 흐름 요약:"""
    )
}

# 6. 프롬프트 선택 + LLM 구성
prompt = prompt_templates.get(question_type, prompt_templates["논문_요약"])
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm  # 최신 LangChain 방식

# 7. 실행
inputs = {"context": context_text}
if "question" in prompt.input_variables:
    inputs["question"] = question_ko

result = chain.invoke(inputs)

# 8. 결과 출력
print(f"[{question_type.upper()}]")
print(result.content)

[연구_흐름]
문형빈 교수님의 연구는 다양한 주제와 분야에서 발전해 왔습니다. 초기에는 주로 기계 학습, 데이터 마이닝, 생물정보학 및 의료 정보학 분야에서 연구를 수행했습니다. 이 시기에는 miRNA-질병 연관성을 예측하는 기계 학습 접근법을 개발하는 등의 연구를 통해 생물 의학 문제 해결에 중점을 두었습니다.

이후, 에너지 경제학 및 정책, 기술 예측, 혁신 관리와 같은 분야로 연구의 초점이 확대되었습니다. 이러한 주제들은 특히 에너지 정책 및 지속 가능한 기술 개발과 관련하여 사회적, 경제적 영향을 분석하는 데 중점을 두고 있습니다. 이와 관련하여, 전기차의 가치 예측 및 대체 연료 차량에 대한 대중의 인식 분석 등과 같은 연구가 포함되었습니다.

가장 최근에는 소비자 행동 분석 및 웨어러블 기술과 같은 고급 기술 제품의 시장성을 평가하는 연구로 연구 관심사가 확장되었습니다. 이 시기에는 소비자의 선호도를 분석하고 새로운 기술 제품의 사회적 및 시장적 영향을 이해하는 연구가 주를 이루고 있습니다.

종합적으로, 문형빈 교수님의 연구는 초기의 생물정보학 및 의료 정보학에서 시작하여, 에너지 정책 및 지속 가능한 기술 개발, 고급 기술 제품의 소비자 행동 분석으로 그 범위가 확장되었습니다. 이는 다양한 사회적, 경제적 및 기술적 문제를 해결하기 위한 학문 간 융합의 예라고 할 수 있습니다.
